In [1]:
from explaneat.data.uci import UCI_WRANGLER
from explaneat.experimenter.experiment import GenericExperiment

import torch
import torch.nn as nn
import torch.optim as optim

import logging
import random


In [2]:
experiment_config_file = './experiment_config.json'
experiment = GenericExperiment(experiment_config_file, confirm_path_creation=False)
logger = experiment.logger

2022-03-27 17:52:15,739 - experimenter - INFO - Validating configuration schema
2022-03-27 17:52:15,740 - experimenter - INFO - Schema validation passed
2022-03-27 17:52:15,740 - experimenter - INFO - Starting to create folder structures
2022-03-27 17:52:15,740 - experimenter - INFO - Experiment folder name is test_experiment_220327T175215_7b7eeaee
2022-03-27 17:52:15,741 - experimenter - INFO - Experiment root path is /Users/mike/dev-mtm/phd-neat-experiments/data/experiments/tests/test_experiment_220327T175215_7b7eeaee
2022-03-27 17:52:15,741 - experimenter - INFO - Creating the root path
2022-03-27 17:52:15,742 - experimenter - INFO - Root path created
2022-03-27 17:52:15,743 - experimenter - INFO - Creating results
2022-03-27 17:52:15,743 - experimenter - INFO - Creating results/interim
2022-03-27 17:52:15,743 - experimenter - INFO - Creating results/final
2022-03-27 17:52:15,744 - experimenter - INFO - Creating configurations
2022-03-27 17:52:15,744 - experimenter - INFO - Creating

In [3]:

USE_CUDA = torch.cuda.is_available()
USE_CUDA = False
device = torch.device("cuda:1" if USE_CUDA else "cpu")
logger.info("Using device: {}".format(device))


2022-03-27 17:52:15,840 - experimenter - INFO - Using device: cpu


# BC Experiment

This experiment (a) test the experimental environment, but is also to evaluate the efficacy of the ExplaNEAT algorithm. Speed is a critical factor, as well as stability of results on population size. Total run time will also be measured

First, we need to set a random seed and a total stopping point in the number of generations

In [4]:
random.seed(experiment.config["random_seed"])
logger.info("random.seed set to {}".format(experiment.config["random_seed"]))

2022-03-27 17:52:15,854 - experimenter - INFO - random.seed set to 42


## Dataset

We are going to work with the Iris dataset, which will be loaded from `sklearn`. We want to characterise the efficacy of the algorithm with regards to a mostly untransformed dataset, so we will only normalise the features

In [5]:
data_wrangler = UCI_WRANGLER(experiment.config['data']['raw_location'],
        experiment.config['data']['raw_data_meta'])

xs are currently
[[2 2 6 ... 0 1 1]
 [3 2 4 ... 1 2 1]
 [3 2 4 ... 0 1 1]
 ...
 [5 1 4 ... 1 0 1]
 [3 1 6 ... 0 1 1]
 [4 1 6 ... 0 1 1]]


In [7]:
data_wrangler.create_train_test_split(experiment.config["train_test_ratio"],
            experiment.config["random_seed"])
data_wrangler.send_train_test_to_device(device)

In [13]:
# xs = torch.from_numpy(xs).to(device)
# ys = torch.from_numpy(ys).to(device)
X_train = torch.from_numpy(X_train).to(device)
X_test = torch.from_numpy(X_test).to(device)
y_train = torch.from_numpy(y_train).to(device)
y_test = torch.from_numpy(y_test).to(device)

Let's have a look at the data we are working with

In [14]:
xs[:5]

array([[-1.64777909,  0.91447105,  0.53223157, -0.45661307,  0.37947332,
         1.29056424, -0.93892436, -0.14681807,  0.5585039 ],
       [-0.65772695,  0.91447105, -0.41913236, -0.45661307,  0.37947332,
        -0.0664261 ,  1.06504852,  0.67651465,  0.5585039 ],
       [-0.65772695,  0.91447105, -0.41913236, -0.45661307,  0.37947332,
        -0.0664261 , -0.93892436, -0.14681807,  0.5585039 ],
       [ 1.32237733, -0.91447105, -0.89481433, -0.45661307,  0.37947332,
        -0.0664261 ,  1.06504852, -0.97015079,  0.5585039 ],
       [-0.65772695,  0.91447105, -2.32186023, -0.45661307,  0.37947332,
        -0.0664261 ,  1.06504852,  1.49984737,  0.5585039 ]])

In [15]:
ys[15:20]

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32)

In [27]:
y_test[:5]

tensor([[0.],
        [1.],
        [0.],
        [1.],
        [1.]])

## Performance metric

The NEAT implementation on which ExplaNEAT extends uses a single function call for evaluating fitness. Although this might be reworked for ExplaNEAT to be able to get consistency between the genome-evaluation and the backprop loss function, that can be reviewed later.

This use `CrossEntropyLoss` from `PyTorch`

In [17]:
def eval_genomes(genomes, config):
    loss = nn.BCELoss()
    loss = loss.to(device)
    for genome_id, genome in genomes.items():
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        preds = []
        for xi in xs:
            preds.append(net.activate(xi))
        genome.fitness = float(1./loss(torch.tensor(preds).to(device), torch.tensor(ys)))

## Base configuration

We are going to create the base configuration according to an external configuration file. Per experiment, we will adjust this, later, but this defines the defaults across all runs.

In [18]:
config_path = "./config-bchard"
base_config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)


We also want to put a hard limit on how long this can go on for.

In [19]:
maxNGenerations = 200

We will create a method to manage the instantiation of a population on the basis of a specific config.

In [20]:
def instantiate_population(config, xs, ys, saveLocation):

    if not os.path.exists(saveLocation):
        os.makedirs(saveLocation)
        
    config.save(os.path.join(saveLocation, 'config.conf'))

    # Create the population, which is the top-level object for a NEAT run.
    p = BackpropPopulation(config, 
                            xs, 
                            ys, 
                            criterion=nn.BCELoss())

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(5, filename_prefix=str(saveLocation) + "checkpoint-" ))
    bpReporter = backprop.BackpropReporter(True)
    p.add_reporter(bpReporter)
    p.add_reporter(ExperimentReporter(saveLocation))
    
    return p

# Experiment 1: Vary population size

The first experiment is going to examine the difference in run time different population sizes. 

In [21]:
epoch_points = [10, 25, 50, 100, 150]
# epoch_points = [10]

In [22]:
base_config.pop_size

50

In [23]:
saveLocationTemplate = './../../data/experiments/bchard/experiment-newalgo-{}-{}/'

## Start the experiment

In [24]:
xs

array([[-1.64777909,  0.91447105,  0.53223157, ..., -0.93892436,
        -0.14681807,  0.5585039 ],
       [-0.65772695,  0.91447105, -0.41913236, ...,  1.06504852,
         0.67651465,  0.5585039 ],
       [-0.65772695,  0.91447105, -0.41913236, ..., -0.93892436,
        -0.14681807,  0.5585039 ],
       ...,
       [ 1.32237733, -0.91447105, -0.41913236, ...,  1.06504852,
        -0.97015079,  0.5585039 ],
       [-0.65772695, -0.91447105,  0.53223157, ..., -0.93892436,
        -0.14681807,  0.5585039 ],
       [ 0.33232519, -0.91447105,  0.53223157, ..., -0.93892436,
        -0.14681807,  0.5585039 ]])

In [25]:
for epochs in epoch_points:
    for iteration_no in range(20):
        my_random_seed += 1
        random.seed(my_random_seed)
        start_time = datetime.now()
        
        print("################################################")
        print("################################################")
        print("Starting epochs {} iteration {}".format(epochs, iteration_no))
        print("Started at {}".format(start_time.strftime("%m/%d/%Y, %H:%M:%S")))
        print("################################################")
        print("################################################")
        
        
        config = deepcopy(base_config)
#         config.pop_size = pop_size
        
        saveLocation = saveLocationTemplate.format(epochs, iteration_no)
        
        p = instantiate_population(config, X_train, y_train, saveLocation)
        # Run for up to nGenerations generations.
        winner = p.run(eval_genomes, maxNGenerations, nEpochs = epochs)
        
        g = p.best_genome

        
        end_time = datetime.now()
        
        p.reporters.reporters[2].save_checkpoint(p.config, p.population, p.species, str(p.generation) + "-final")  
        
        winner_net = neat.nn.FeedForwardNetwork.create(winner, config)

        results = []
        for xi, xo in zip(xs, ys):
            output = winner_net.activate(xi)
            results.append([xi, xo, output])

        df = pd.DataFrame(results)
        df.to_csv(os.path.join(saveLocation, 'results.csv'))
        
        ancestry = p.reporters.reporters[3].trace_ancestry_of_species(g.key, p.reproduction.ancestors) 

        ancestors = {
            k: v['genome'] for k, v in p.reporters.reporters[3].ancestry.items()
        }
        
    
        
        ## Save all of these to disc
        filename = 'fullStatus.xplnt'
        print("Saving checkpoint to {0}".format(filename))

        with gzip.open(os.path.join(saveLocation, filename), 'w', compresslevel=5) as f:
            data = (p, g, ancestry, ancestors, random.getstate())
            pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)
            
            
        with gzip.open(os.path.join(saveLocation, 'train_test_data.pkl'), 'w', compresslevel=5) as f:
            train_Test = (X_train, X_test, y_train, y_test)
            pickle.dump(train_Test, f, protocol=pickle.HIGHEST_PROTOCOL)
#         visualize.create_ancestry_video(p.config, 
#                                         g, 
#                                         ancestry, 
#                                         ancestors, 
#                                         p.reporters.reporters[1], 
#                                         pathname=saveLocation)
        print("################################################")
        print("################################################")
        print("Have finished epochs {} iteration {}".format(epochs, iteration_no))
        print("Started at {}".format(start_time.strftime("%m/%d/%Y, %H:%M:%S")))
        print("The time is {}".format(end_time.strftime("%m/%d/%Y, %H:%M:%S")))
        print("################################################")
        print("################################################")
    

################################################
################################################
Starting epochs 10 iteration 0
Started at 03/13/2022, 15:37:32
################################################
################################################
The function - generationStart - has just started at 1647139052.888087

 ****** Running generation 0 ****** 

The function - generationStart - took 0.0002110004425048828 seconds to complete
The function - pre_backprop - has just started at 1647139052.8883111
The function - pre_backprop - took 6.079673767089844e-05 seconds to complete
The function - backprop - has just started at 1647139052.8884401
about to start backprop with 10 epochs
The function - backprop - took 0.020262956619262695 seconds to complete


RuntimeError: Found dtype Float but expected Double